# Using Writer with AWS Strands Agents

This notebook demonstrates how to build an agent using the Strands Agents SDK with [WRITER’s models](https://dev.writer.com/home/integrations/strands#available-models).

Strands Agents SDK from AWS is an open-source framework that enables developers to build and deploy AI agents using a model-driven approach. This integration allows you to use Writer models within the Strands agent ecosystem, from local development to production deployment.

## Prerequisites

Before getting started, you'll need:

- A [Writer AI Studio](https://app.writer.com/register) account
- An API key, which you can obtain by following the [API Quickstart](https://dev.writer.com/api-guides/quickstart)

## Setup
To use Writer models with Strands Agents, install the optional Writer dependency along with the [Strands Agents Tools package](https://github.com/strands-agents/tools):

In [ ]:
%pip install 'strands-agents[writer]'
%pip install strands-agents-tools

Next, set the `WRITER_API_KEY` environment variable. It is recommended to set this value in a `.env` file in the root of the project; however, this tutorial sets it directly as an environment variable if a `.env` file is not available.

In [ ]:
import getpass
import os

if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API key: ")


## Usage

Initialize a Strands agent using a Palmyra model from WRITER.

### Choose a Writer model

Import `WriterModel` from the Strands integration and specify a [model ID](https://dev.writer.com/home/integrations/strands#available-models) (for example, `palmyra‑x5`, `palmyra‑creative`, `palmyra‑med`).

In [ ]:
from strands import Agent
from strands.models.writer import WriterModel

writer_model = WriterModel(
    model_id="palmyra-x5"  # or other Palmyra model like palmyra-creative, palmyra-fin, etc.
)

### Configure tools

Define a list of tools to pass to the agent, such as a calculator or HTTP request tool.

> The `calculator` tool shown in this example is a prebuilt tool from the [`strands_tools` package](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/example-tools-package/). You can use existing tools from `strands_tools`, or define custom tools—see the [Python Tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/python-tools/?h=python+tools) documentation for guidance.



In [ ]:
from strands_tools import calculator

tools = [calculator]


### Initialize the agent

Initialize the agent with the configured tools, `writer_model`, and a system prompt that defines the agent’s role, tone, output format, and how it should use tools.

> **Best practice:** Use a clear and specific system prompt to define your agent’s role and expected behavior. If you include tools, be explicit about when and how the agent should use them. For many use cases, a single agent with one model is sufficient and keeps the architecture simple.

In [ ]:
SYSTEM_PROMPT = """
You are a helpful enterprise assistant. Use your knowledge and available tools (if provided)
to answer user queries accurately and helpfully.
"""

agent = Agent(
    model=writer_model,
    tools=tools,         # or [] if no tools
    system_prompt=SYSTEM_PROMPT
)


### Run the agent

Once you’ve finished importing and initializing the WRITER provider in Strands Agents, you can ask the agent a question.

In [ ]:
response = agent("What is 2+2")
print(response)


## Example: advanced multi-agent research and content creation

This code provides an **extended example** of a multi-agent workflow 
using the Strands SDK. It goes beyond a basic demonstration to showcase a more 
detailed, real-world application scenario involving multiple specialized agents:

1. **Research Agent**: Leveraging `palmyra-x5` to perform comprehensive web-based research, 
   gather verified information, and summarize insights.

2. **Content Agent**: Utilizing `palmyra-creative` to craft engaging, platform-optimized 
   social media content based on the research findings.

This extended example covers:

- Importing and configuring necessary modules  
- Initializing and customizing models for each agent  
- Creating agents with specialized roles and workflows  
- Orchestrating a multi-step pipeline for research and content creation  
- Passing intermediate outputs between agents in a multi-step workflow

## Setup
Import the required modules to define agents, configure WRITER models, and enable tool-based HTTP requests:

In [ ]:
from strands import Agent
from strands.models.writer import WriterModel
from strands_tools import http_request

## Initialize the models

- `palmyra-creative` → for creative social content.
- `palmyra-x5` → for accurate research and factual lookups.


In [ ]:
# Initialize the creative content model
content_model = WriterModel(
    model_id="palmyra-creative",
)

# Initialize the research model
research_model = WriterModel(
    model_id="palmyra-x5",
)


## Define system prompts

- **Content agent prompt:** instructs the agent to write engaging, platform-specific posts.
- **Research agent prompt:** instructs the agent to gather and summarize factual information from the web.


In [ ]:
# Content agent system prompt
CREATIVE_ASSISTANT_SYSTEM_PROMPT = """
You are the Creative Content Assistant. Your job is to craft imaginative, engaging, and on‑brand content for both X/Twitter and LinkedIn.
Always use vivid language, a friendly tone, and creative flair to delight and inspire the reader.

For each request:
- Write distinct content tailored to each platform's tone and audience
- Label each section clearly as 'LinkedIn' and 'X/Twitter'
- Briefly list the steps or reasoning you used to craft the content
"""

# Research agent system prompt
RESEARCH_ASSISTANT_SYSTEM_PROMPT = """
You are a Researcher Assistant that gathers information from the web. 
1. Determine if the input is a research query or factual claim 
2. Use your research tools (http_request, retrieve) to find relevant information 
3. Include source URLs and keep findings under 500 words
"""


## Create agents

- **Research Agent** uses `http_request` for live web research.  
- **Content Agent** generates polished social content.


In [ ]:
# Create the research agent
research_agent = Agent(
    system_prompt=RESEARCH_ASSISTANT_SYSTEM_PROMPT,
    callback_handler=None,
    model=research_model,
    tools=[http_request]
)

# Create the content agent
content_agent = Agent(
    system_prompt=CREATIVE_ASSISTANT_SYSTEM_PROMPT,
    callback_handler=None,
    model=content_model
)

## Define the workflow function

This function orchestrates both agents:

1. **Research step:** gather factual information.  
2. **Content step:** generate X/Twitter and LinkedIn posts based on the research.


In [ ]:
def run_content_workflow(user_input):
    # Step 1: Research
    researcher_response = research_agent(
        f"Research: '{user_input}'. Use your available tools to gather information from reliable sources.",
    )
    research_findings = str(researcher_response)

    # Step 2: Generate content
    content = content_agent(
        f"Create X/Twitter and LinkedIn social post content based on these findings about '{user_input}': \n\n{research_findings}",
    )

    return content


## Run the workflow

Test the workflow using a real announcement from Strands Agents 1.0.

In [ ]:
response = run_content_workflow(
    "Write a creative launch announcement for Strands Agents 1.0 from their announcement here: https://aws.amazon.com/blogs/opensource/introducing-strands-agents-1-0-production-ready-multi-agent-orchestration-made-simple/"
)
print(response)


## Example: Writer MCP + Strands agents  

This recipe demonstrates how to combine:

- **Writer MCP Server** (via `writer-sdk-mcp`)
- **Strands Agents**
- **WriterModel (Palmyra-x5)**
- **MCP Tools over stdio**

to orchestrate multi-step research and content generation workflows.


## Install dependencies

Install Strands (with Writer support) and the Strands Agent Tools package:

In [ ]:
%pip install 'strands-agents[writer]' strands-agents-tools

## Building a custom MCP client with Strands

In [ ]:
import os
from strands import Agent
from strands.models.writer import WriterModel
from strands.tools.mcp import MCPClient
from mcp import stdio_client, StdioServerParameters



def log_server_init_and_requests(client_factory):
    """Wrap an MCP client to log server start and requests."""
    class LoggingMCPClient:
        def __init__(self):
            self._client = client_factory()

        def __enter__(self):
            print("[LOG] Starting MCP server...")
            self._client.__enter__()
            print("[LOG] MCP server started.")
            return self

        def __exit__(self, exc_type, exc_val, exc_tb):
            self._client.__exit__(exc_type, exc_val, exc_tb)

        def __getattr__(self, name):
            orig_attr = getattr(self._client, name)
            if callable(orig_attr):
                def hooked(*args, **kwargs):
                    print(f"[LOG] MCP request: {name}, args={args}, kwargs={kwargs}")
                    result = orig_attr(*args, **kwargs)
                    print(f"[LOG] MCP Server starting with tools: {',*\n'.join([el.tool_name for el in result])}")
                    return result
                return hooked
            return orig_attr
    return LoggingMCPClient()


# Connect to the Writer MCP server via stdio
writer_mcp_client = log_server_init_and_requests(lambda: MCPClient(lambda: stdio_client(
    StdioServerParameters(
        command="npx",
        args=["-y", "writer-sdk-mcp@latest"],
        env={"WRITER_API_KEY": os.environ["WRITER_API_KEY"]}
    )
)))

def content_workflow_client(user_request: str):
    """
    Custom MCP client that orchestrates complex content workflows
    using Writer MCP tools.
    """
    with writer_mcp_client:
        tools = writer_mcp_client.list_tools_sync()
        agent = Agent(
            model=writer_model,
            tools=tools,
            system_prompt="""You are a content workflow assistant that can:
            - Upload and analyze files
            - Generate content using Writer's models
            - Query Knowledge Graphs for research
            - Analyze images and extract insights
            - Chain multiple operations together seamlessly
            Use the available Writer MCP tools to fulfill user requests efficiently."""
        )
        return agent(user_request)


## Example workflow request

In [ ]:
response = content_workflow_client(
    "Research and create content about sustainable fashion trends, "
    "including a blog post outline and social media content ideas."
)

print("Research & Content:", response)